In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 

train_df = pd.read_csv('/kaggle/input/training/train.csv',skiprows=0)

Y = train_df.filter(like='Blend')

X = train_df.drop(Y.columns , axis = 1)


In [42]:
def feature_expansion(X):

    x_engg = X.copy()
    new_features = []
    
    for i in range(1,6):
        fractionc = f"Component{i}_fraction"
        proportionc = X[fractionc]
        propertyc  = f"Component{i}_Property"
        ComponentPropertiesc = X.filter(like=propertyc)  
        partial_componets = proportionc.values.reshape(-1, 1) * ComponentPropertiesc
    
        partial_componets.columns = [f'weighted_C{i}_P{j}' for j in range(1, 11)]
        
        new_features.append(partial_componets)
    
    X_final = pd.concat([x_engg] + new_features, axis=1 )
    return X_final

In [43]:
def cost(y_test,prediction):
    mape_cost = mean_absolute_percentage_error(y_test, prediction)
    reference_cost = 2.72
    leaderboard_score = 100 - (90 * mape_cost) / reference_cost
    final_score = max(10, leaderboard_score)
    return f"Final Validation MAPE: {mape_cost:.4f},Final Score with Linear Regression: {final_score:.2f}"
    

In [44]:
x_train, x_test  = train_test_split(X,test_size=0.2)
y_train , y_test = train_test_split(Y,test_size=0.2)


poly_transformer = PolynomialFeatures(degree=2, include_bias=False)

x_train = poly_transformer.fit_transform(x_train)

model = LinearRegression()

model.fit(x_train,y_train)


x_test = poly_transformer.transform(x_test)




In [45]:
p = model.predict(x_test)

In [46]:

print(f"score:{model.score(x_test, y_test)}")

print(cost(y_test,p))

score:-32.188386779764556
Final Validation MAPE: 75.6577,Final Score with Linear Regression: 10.00
